# Classificador Naive Bayes

Atualmente, o algoritmo se tornou popular na área de Aprendizado de Máquina (Machine Learning) para **categorizar textos baseado na frequência das palavras usadas**, e assim pode ser usado para identificar se determinado e-mail é um SPAM ou sobre qual assunto se refere determinado texto, por exemplo.

Por ser muito simples e rápido, possui um desempenho relativamente maior do que outros classificadores. Além disso, o Naive Bayes só precisa de um pequeno número de dados de teste para concluir classificações com uma boa precisão.

A principal característica do algoritmo, e também o motivo de receber “naive” (ingênuo) no nome, é que **ele desconsidera completamente a correlação entre as variáveis (features)**. Ou seja, se determinada fruta é considerada uma “Maçã” se ela for “Vermelha”, “Redonda” e possui “aproximadamente 10cm de diâmetro”, o algoritmo não vai levar em consideração a correlação entre esses fatores, tratando cada um de forma independente.

In [399]:
from functools import lru_cache
from nltk.corpus import floresta
from textblob.classifiers import NaiveBayesClassifier
from string import digits, punctuation
from pprint import pprint
from random import shuffle
import stop_words
import textblob
import re
import nltk
import requests
import csv
import math

## Dados

Neste notebook vamos tentar aplicar o classificador Naive Bayes para classificar discursos dos parlamentares da Câmara dos Deputados proferidos em plenário. Os dados utilizados para análise foram obtidos através do [Babel](https://dev.babel.labhackercd.leg.br/), um repositórios de dados de manifestações políticas, e podem ser acessados pelo *endpoint*: https://dev.babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=2 .

In [400]:
response = requests.get('https://dev.babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=2')
data = response.json()['results']
response = requests.get('https://dev.babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=2&page=10')
data += response.json()['results']
response = requests.get('https://dev.babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=2&page=20')
data += response.json()['results']

speeches = []
for speech in data:
    for attr in speech['attrs']:
        if attr['field'] == 'original':
            speeches.append(attr['value'])
            break

print(len(speeches), 'discursos coletados')

600 discursos coletados


In [401]:
# Método para limpar texto do discurso tirando as notas do taquigrafo
def clear_speech(text):
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[OA] SRA?[\w\s.]+-', '', text)
    text = re.sub(r'PRONUNCIAMENTO[\sA-Z]+\s', '', text)
#     text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s[\.\"]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[Vv]\.[Ee][Xx][Aa]\.', 'v.exa', text)
    text = re.sub(r'[Aa][Rr][Tt]\.', 'art', text)
    text = re.sub(r'[Ss][Rr][Ss]?\.', 'sr', text)
    text = re.sub(r'[Ss][Rr][Aa][Ss]?\.', 'sr', text)
    text = re.sub(r'\d', '', text)
    return text.strip()

cleaned_text = clear_speech('. '.join(speeches))
blob = textblob.TextBlob(cleaned_text)

### Sentenças

Para realizar a classificação de um discurso, ele será divido em sentenças, utilizando a biblioteca TextBlob, e cada sentença será classificada individualmente e o conjunto de sentenças classificadas (uma sentença não precisa ser, necessariamente, classificada) definirá a classificação de um discurso.

In [465]:
print(blob.sentences[23], end='\n\n')
print(blob.sentences[233], end='\n\n')
print(blob.sentences[100], end='\n\n')

Presidente!

O PSDB é a favor da votação.. sr Presidente Deputado Eduardo Cunha, sr e sr Parlamentares, a PEC  é um acerto da Federação, ela faz parte do pacto federativo.

Pela primeira vez, nós teremos uma estação de tratamento de água, haverá produção de água com qualidade, para a população da cidade de Óbidos.



## Stopwords

*Stopwords* (ou palavras de parada – tradução livre) são palavras que podem ser consideradas irrelevantes para o conjunto de resultados a ser exibido em uma busca realizada em uma search engine. Exemplos: as, e, os, de, para, com, sem, foi. Esse conjunto de palavras pode variar em função do contexto em que será utilizado.

Para esse notebook serão utilizadas as palavras fornecidas pela biblioteca NLTK, além de outras palavras selecionadas de acordo com os textos utilizados.

In [403]:
stemmer = nltk.RSLPStemmer()

In [404]:
stopwords = stop_words.get_stop_words('portuguese') + [
    'presidente', ',', '.', '...', 'é', 'questão', 'art', 'ordem', 'v.exa', ':', 'governo', 'sr', 'agência', 'º',
    'aqui', 'vai', 'artigo', '§', 'neste', 'vamos', 'agora', "''", 'fazer', 'mesa', 'ainda', 'porque', 'trata',
    'estrutura', 'sobre', 'então', 'todos', 'obstrução', 'votação', 'presença', 'deputados', 'vou', 'brasil',
    'discutir', 'vigência', 'colocar', 'regimento', 'momento', ';', 'dois', 'dessa', 'medida', 'proposta',
    'casa', 'matéria', 'queria', 'assim', 'possamos', 'microfone', 'certeza', 'hoje', 'profissional', 'deixar',
    'provisória', 'ora', 'base', 'importante', 'veto', 'fala', '!', 'ministério', 'aumento', 'inciso',
    'manifestação', 'xiii', 'diálogo', 'podemos', 'apenas', 'poder', 'efeitos', 'pode', 'acordo', 'solicitação',
    'reflexão', '?', 'ausência', 'aprovada', 'lideranças', 'dizer', 'bancada', 'portanto', 'peço', 'recolher',
    'prática', 'pois', 'democracia', 'milhões', 'bilhões', 'melhoria', 'atividade', 'claro', 'saber', 'dar',
    'avanço', 'condições', 'desastre', 'especialmente', 'exatamente', 'política', 'vezes', 'fazê-lo', 'têm',
    'derrubar', 'precisa', 'custo', 'necessária', 'cláusula', 'proposição', '-', 'palavra', 'tempo', 'segundos',
    'fez', 'necessário', 'zero', 'interesse', 'srs', 'sr', 'sras', 'sra', 'deputado', 'presidente', 'é', 'nº',
    's.a.', 'v.exa.', 'v.exa', '#', 'anos', 'º', 'exa', 'mesa', 'legislatura', 'sessão', 'maioria', 'seguinte',
    'mandato', 'bilhões', 'quilômetros', 'ª', 'parabéns', 'membros', 'convido', 'usual', 'biênio',
    'brasil', 'palavra', 'discussão', 'período', 'início', 'pronunciamento', 'suplente', 'atividade', 'ação',
    'ações', 'daqueles', 'diferenças', 'pasta', 'milhares', 'srªs', 'emenda', 'àqueles', 'tamanha', 'mês',
    'capaz', 'km', 'modelo', 'tarefas', 'colegas', 'programa', 'voz', 'meios de comunicação', 'pronunciamento',
    'casa', 'sessão', 'deliberativa', 'solene', 'ordinária', 'extraordinária', 'encaminhado', 'orador', 'tv',
    'divulgar', 'deputado', 'parlamento', 'parlamentar', 'projeto', 'proposta', 'requerimento', 'destaque',
    'veto', 'federal', 'câmara', 'senado', 'congresso', 'nacional', 'país', 'estado', 'brasil', 'lei',
    'política', 'povo', 'voto', 'partido', 'liderança', 'bancada', 'bloco', 'líder', 'lider', 'frente',
    'governo', 'oposição', 'presença', 'presente', 'passado', 'ausência', 'ausencia', 'ausente', 'obstrução',
    'registrar', 'aprovar', 'rejeitar', 'rejeição', 'sabe', 'matéria', 'materia', 'questão', 'ordem', 'emenda',
    'sistema', 'processo', 'legislativo', 'plenário', 'pedir', 'peço', 'comissão', 'especial', 'permanente',
    'apresentar', 'encaminhar', 'encaminho', 'orientar', 'liberar', 'apoiar', 'situação', 'fato', 'revisão',
    'tempo', 'pauta', 'discutir', 'discussão', 'debater', 'retirar', 'atender', 'colegas', 'autor', 'texto',
    'medida', 'união', 'república', 'audiência', 'audiencia', 'público', 'publico', 'reunião', 'agradecer',
    'solicitar', 'assistir', 'contrário', 'favorável', 'pessoa', 'comemorar', 'ato', 'momento', 'diretora',
    'possível', 'atenção', 'agradeço', 'naquele', 'necessárias', 'presidenta', 'compromisso', 'geradas',
    'primeiro', 'simplesmente', 'ideal', 'argumento', 'i', 'válido', 'envolvidos', 'nesse', 'aspecto',
    'existentes', 'normativo', 'irá', 'nada', 'melhor', 'esperarmos', 'pouco', 'resolvermos', 'problema',
    'postura', 'faltas', 'declara', '%', 'grande', 'dia', 'obrigado', 'agradeço', 'agradecido', 'população',
    'maior', 'cada', 'bem', 'mundo', 'desta', 'mil', 'sendo', 'outros', '$', '!', '@', '#', '&', '(', ')', 'sim',
    'r', 'sempre', 'além', 'semana', 'relação', 'onde', 'meio', 'inclusive', 'lá', 'vem', 'menos', 'menor',
    'qualquer', 'desde', 'ontem', 'hoje', 'exemplos', 'exemplo', 'tão', 'fim', 'janeiro', 'fevereiro', 'março',
    'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro', 'alguns', 'tudo',
    'durante', 'gostaria', 'três', 'conta', 'feito', 'através', 'antes', 'depois', 'verdade', 'bom', 'quase',
    'setor', 'aí', 'disse', 'principalmente', 'final', 'vão', 'coisa', 'ver', 'sentido', 'nova', 'vários', 'novo',
    'nenhuma', 'quanto', 'infelizmente', 'felizmente', 'número', 'duas', 'dois', 'tanto', 'acho', 'achar',
    'enquanto', 'deve', 'apelo', 'papel', 'últimos', 'faço', 'fazer', 'garantir', 'garantia', 'fica', 'obrigado',
    'obrigado..', 'assunto', 'sido', 'vir', 'incrementar', 'central', 'aproximado', 'aproximadamente',
    'hipotética', 'hipotese', 'hipótese', 'média', 'superior', 'superiores', 'gerais', 'venha', 'minas'
]
stopwords += [x for x in punctuation]
stem_stopwords = [stemmer.stem(word) for word in stopwords]

In [405]:
@lru_cache()
def simplify_tag(tag):
    if "+" in tag:
        return tag[tag.index("+") + 1:]
    else:
        return tag

floresta_twords = floresta.tagged_words()
for (word, tag) in floresta_twords:
    tag = simplify_tag(tag)
    words = word.casefold().split('_')
    if tag not in ('adj', 'n', 'prop', 'nprop', 'est', 'npro'):
        stopwords += [stemmer.stem(word) for word in words]

stopwords = list(set(stopwords))

In [412]:
all_words = nltk.tokenize.word_tokenize(cleaned_text)
words = [word.casefold() for word in all_words if stemmer.stem(word) not in stem_stopwords]
dist = nltk.FreqDist(words)
print('Palavras mais comuns')
print(dist.most_common(100))

Palavras mais comuns
[('rio', 422), ('segurança', 242), ('trabalho', 228), ('municípios', 221), ('educação', 220), ('sociedade', 189), ('saúde', 188), ('recursos', 182), ('cidade', 170), ('intervenção', 167), ('vida', 160), ('trabalhadores', 156), ('direito', 147), ('crise', 141), ('reforma', 137), ('município', 136), ('pec', 130), ('petrobras', 123), ('justiça', 122), ('ministro', 120), ('reais', 119), ('defesa', 119), ('social', 118), ('constituição', 117), ('lotéricos', 116), ('lula', 113), ('prefeito', 112), ('sul', 111), ('dilma', 110), ('dinheiro', 107), ('empresas', 105), ('civil', 104), ('paulo', 103), ('previdência', 101), ('polícia', 99), ('respeito', 98), ('luta', 95), ('família', 94), ('deus', 92), ('econômica', 91), ('tribunal', 89), ('comunicação', 89), ('violência', 89), ('renda', 88), ('corrupção', 88), ('desenvolvimento', 87), ('pt', 87), ('boa', 87), ('escola', 85), ('decisão', 84), ('escolas', 84), ('professores', 83), ('responsabilidade', 83), ('serviço', 83), ('car

## Naive Bayes

Para esse notebook, usaremos a implementação do TextBlob para classificar as sentenças. No TextBlob, podemos definir um método de extração de atributos (*feature extraction*) que são utilizados para comparar dois textos. Esse método deve receber como parâmetro o texto, pode receber um segundo argumento que contem os dados de treinamento e deve retornar um dicionário de atributos do texto recebido

In [413]:
regex = re.compile('[%s]' % re.escape(punctuation))

def freq_feature_extractor(document):
    document = regex.sub(' ', document.casefold())
    tokens = nltk.tokenize.word_tokenize(document)
    tokens = [stemmer.stem(token) for token in tokens if stemmer.stem(token) not in stem_stopwords]
    dist = nltk.FreqDist(tokens)
    
    features = {
        stem: True
#         stem: dist.freq(stem)
        for stem, _ in dist.most_common()
    }
    return features

freq_feature_extractor('Calcula-se que a taxação de grandes patrimônios poderia render aproximadamente  bilhões de reais por ano se aplicada uma alíquota média de % sobre os bens das pessoas, em uma simulação hipotética, sobre valores superiores a  milhão de reais.')

{'real': True,
 'calcul': True,
 'tax': True,
 'patrimôni': True,
 'rend': True,
 'aplic': True,
 'alíquot': True,
 'simul': True,
 'val': True}

## Dados de treinamento

A planilha de treinamento está disponível no [Google Drive](https://docs.google.com/spreadsheets/d/1qmJjRexlSUYOAN12IY_82NyOUaLZlbBsoeYRuulEjFE/edit?usp=sharing).

In [414]:
LABELS_RELATION = {
    0: 'none',
    1: 'adm-publica',
    2: 'agricultura-pecuaria-pesca-extrativismo',
    3: 'arte-cultura-religiao',
    4: 'cidades-desenvolvimento-urbano',
    5: 'ciencia-tecnologia-inovacao',
    6: 'ciencias-exatas',
    7: 'ciencias-sociais',
    8: 'comunicacoes',
    9: 'defesa-seguranca',
    10: 'direito-civil',
    11: 'direito-constitucional',
    12: 'direito-consumidor',
    13: 'direito-justica',
    14: 'direito-penal',
    15: 'direitos-humanos-minorias',
    16: 'economia',
    17: 'educacao',
    18: 'energia-recursos-hidricos-minerais',
    19: 'esporte-lazer',
    20: 'estrutura-fundiaria',
    21: 'financas-publicas-orcamento',
    22: 'homenagens-datas-comemorativas',
    23: 'industria-comercio-servicos',
    24: 'meio-ambiente-desenvolvimento-sustentavel',
    25: 'politica-partidos-eleicoes',
    26: 'previdencia-assistencia-social',
    27: 'processo-legislativo-atuacao-parlamentar',
    28: 'relacoes-internacionais-comercio-exterior',
    29: 'saude',
    30: 'trabalho-emprego',
    31: 'turismo',
    32: 'viacao-transporte-mobilidade',
}

In [456]:
theme_data = []
content_data = []

with open('naive-bayes-train.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    next(reader)
    for row in reader:
        for idx, category in enumerate(row[1:-1]):
            if category != '' and idx > 0:
                theme_data.append((row[0], LABELS_RELATION[idx]))
                content_data.append((row[0], 'content'))
            elif category != '' and idx == 0:
                content_data.append((row[0], 'useless'))

shuffle(content_data)
content_data_size = len(content_data)
content_train = content_data[:math.floor(content_data_size * 0.7)]
content_test = content_data[math.floor(content_data_size * 0.7):]
print('Treinamento de conteúdo: {} sentenças'.format(len(content_train)))
print('Teste de conteúdo: {} sentenças'.format(len(content_test)))

shuffle(theme_data)
theme_data_size = len(theme_data)
theme_train = theme_data[:math.floor(theme_data_size * 0.7)]
theme_test = theme_data[math.floor(theme_data_size * 0.7):]
print('Treinamento de temas: {} sentenças'.format(len(theme_train)))
print('Teste de temas: {} sentenças'.format(len(theme_test)))

Treinamento de conteúdo: 210 sentenças
Teste de conteúdo: 90 sentenças
Treinamento de temas: 147 sentenças
Teste de temas: 63 sentenças


In [457]:
content_classifier = NaiveBayesClassifier(content_train, feature_extractor=freq_feature_extractor)
content_classifier.accuracy(content_test)

0.7777777777777778

In [458]:
theme_classifier = NaiveBayesClassifier(theme_train, feature_extractor=freq_feature_extractor)
theme_classifier.accuracy(theme_test)

0.031746031746031744